In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [2]:


colors_df = pd.DataFrame(data=[['red'],['blue'],['green'],['blue']], columns=['color'])
print('Before One Hot Encoding:')
display(colors_df)

one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_encoder.fit(colors_df)

colors_df_encoded = one_hot_encoder.transform(colors_df)
colors_df_encoded = pd.DataFrame(data=colors_df_encoded, columns=one_hot_encoder.categories_)
print('\nAfter One Hot Encoding:')
display(colors_df_encoded)

Before One Hot Encoding:


,color
0,red
1,blue
2,green
3,blue



After One Hot Encoding:


,blue,green,red
0,0.0,0.0,1.0
1,1.0,0.0,0.0
2,0.0,1.0,0.0
3,1.0,0.0,0.0


In [3]:
#  https://stackoverflow.com/questions/57518057/how-can-i-convert-an-image-from-pixels-to-one-hot-encodings

image_colors = [
    (0, 0, 0),  # black
    (127, 127, 127),  # grey
    (255, 127, 255),  # pink
]
path = 'ramp-data/metric_data/onehot_source/chips/OAM-309612-261633-19.tif'

In [4]:
import tensorflow as tf


In [5]:
# Create a "color reference" tensor from image_colors
color_reference = tf.cast(tf.constant(image_colors), dtype=tf.uint8)

# Load the image and obtain tensor with one-hot values
image = tf.io.decode_image(tf.io.read_file(path), channels=3)
comp = tf.equal(image[..., None, :], color_reference)
one_hot = tf.cast(tf.reduce_all(comp, axis=-1), dtype=tf.float32)

2024-03-01 11:19:35.449161: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-01 11:19:35.903747: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 561 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0001:00:00.0, compute capability: 7.5


InvalidArgumentError: Unknown image file format. One of JPEG, PNG, GIF, BMP required. [Op:DecodeImage]

In [6]:
from keras.utils.np_utils import to_categorical
categorical_labels = to_categorical(image_colors, num_classes=None)

In [7]:
categorical_labels

array([[[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.]]], dtype=float32)

In [13]:
# one more trial https://stackoverflow.com/questions/69942647/sparse-categorical-crossentropy-shape-problem-with-keras
num_class=None
def get_img_and_onehot_class(img_path, class):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_jpeg(img, channels=3) # this function doens't exist for tiff files, only experimental
    """ Other preprocessing of image."""
    return img, tf.one_hot(class, num_class)


In [15]:

# res = tf.one_hot(indices=[0, 3], depth=4)
# with tf.Session() as sess:
#     print sess.run(res)



SyntaxError: invalid syntax (369728695.py, line 3)

### Tryal with actual image

In [24]:
import matplotlib.pyplot as plt
from osgeo import gdal
import os
import numpy as np

In [29]:
image_path = 'ramp-data/metric_data/onehot_source/binarymasks/OAM-309606-261636-19.mask.tif'

In [35]:
# image_chip = gdal.Open(r'ramp-data/metric_data/onehot/chips/OAM-309612-261633-19.tif') # 3 bands
image_chip = gdal.Open(r'ramp-data/metric_data/onehot_source/binarymasks/OAM-309606-261636-19.mask.tif')

In [36]:
import tensorflow as tf
n_classes = 4
ohe_labels = tf.one_hot(image_chip, n_classes)

ValueError: Attempt to convert a value (<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f3b4b163870> >) with an unsupported type (<class 'osgeo.gdal.Dataset'>) to a Tensor.

In [30]:
categorical_chip = to_categorical(image_path, num_classes=None)
# doesn't work

ValueError: invalid literal for int() with base 10: 'ramp-data/metric_data/onehot_source/binarymasks/OAM-309606-261636-19.mask.tif'

let's try convert the tif to png?

In [13]:
# %run python3 -m convert_tif_to_png --file ramp-data/metric_data/onehot_source/chips/OAM-309612-261633-19.tif
#  we can convert to png, but in our case it wouldn't make sense, as we lose the bands info

In [18]:
# https://stackoverflow.com/questions/68170720/tiff-workaround-for-tensorflow
import cv2
def parse_with_opencv(image_path):
    return cv2.imread(image_path.decode('UTF-8'))

img_path = [path]

ds = tf.data.Dataset.from_tensor_slices(img_path).map(
    lambda x: tf.numpy_function(parse_with_opencv, [x], Tout=tf.uint8)
)
# doesn't work:
# Unbatching a tensor is only supported for rank >= 1
# works after adding the [] to the image path, from here:
# https://stackoverflow.com/questions/55560620/valueerror-unbatching-a-tensor-is-only-supported-for-rank-1


In [ ]:
# https://stackoverflow.com/questions/67969346/how-to-create-tf-dataset-from-geotiff-files-using-custom-map-function
ds = tf.data.Dataset.from_tensor_slices([path])
ds = ds.map(lambda x: tf.py_function(parse_image, [x], [tf.float32])).batch(8)
# again Unbatching a tensor is only supported for rank >= 1
# yet doesn't work with the [], we can use the one above

changing all files in folder

In [20]:
import glob

In [21]:
folder_images = "ramp-data/metric_data/onehot_source/chips/"
img_list = [fille for fille in glob.glob(f'{folder_images}*.tif')]
output_folder = "ramp-data/metric_data/onehot/binarymasks/"

In [22]:
img_list

['ramp-data/metric_data/onehot_source/chips/OAM-309606-261636-19.tif',
 'ramp-data/metric_data/onehot_source/chips/OAM-309612-261632-19.tif',
 'ramp-data/metric_data/onehot_source/chips/OAM-309613-261632-19.tif',
 'ramp-data/metric_data/onehot_source/chips/OAM-309612-261633-19.tif',
 'ramp-data/metric_data/onehot_source/chips/OAM-309613-261633-19.tif',
 'ramp-data/metric_data/onehot_source/chips/OAM-309614-261632-19.tif',
 'ramp-data/metric_data/onehot_source/chips/OAM-309614-261633-19.tif']

In [23]:
for img in img_list:
    print(img)
    cat_chip = to_categorical(img, num_classes=None)
    


ramp-data/metric_data/onehot_source/chips/OAM-309606-261636-19.tif


ValueError: invalid literal for int() with base 10: 'ramp-data/metric_data/onehot_source/chips/OAM-309606-261636-19.tif'

In [ ]:
img_list

In [ ]:
cat_chip